In [1]:
import argparse
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torch.optim as optim
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
import torch.nn.functional as F
import tensorflow as tf
import random

In [2]:
# import np vectors used in classification
### NOT ACTUALLY USED
### perhaps better syntax tho
avis = []
lab = []

labels = [["sinker",0], ["curve", 1]]
labels2 = [["ball", 1], ["strike", 0]]

split = int(len(avis)*3/4)

x_train, x_test, y_train, y_test = map(
    torch.tensor, (avis[:split], avis[split:], lab[:split], lab[split:]))

y_names = labels[:][0]
labels = labels[:][1]

In [3]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, padding=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=2, padding=2)
        self.bn2 = nn.BatchNorm2d(32)


        self.conv5 = nn.Conv2d(32, 16, kernel_size=3, stride=2, padding=2)
        self.bn5 = nn.BatchNorm2d(16)
        self.conv6 = nn.Conv2d(16, 4, kernel_size=3, stride=2, padding=2)
        self.bn6 = nn.BatchNorm2d(4)

        self.fc1 = nn.Linear(3772, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 2)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, 0.3, training=self.training)

        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.max_pool2d(x, 2)
        x = F.dropout(x, 0.3, training=self.training)

        x = x.view(x.size(0), -1)  # Flatten layer
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.3, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [4]:
import cv2
import numpy as np

def extract_frames(video_path, frame_indices):
    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if the video file is opened successfully
    if not cap.isOpened():
        print("Error: Unable to open video file.")
        return

    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    rev_idxs = (total_frames-1)*np.ones(len(frame_indices))-np.array(frame_indices)
    # Extract frames at specified indices
    for idx in rev_idxs:
        # Set the frame position
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)

        # Read the frame
        ret, frame = cap.read()

        # Check if the frame is read successfully
        if ret:
            # Save the frame
            # cv2.imwrite(f"{output_folder}/frame_{idx}.jpg", frame)
            # print(f"Frame {idx} saved successfully.")
            break


        else:
            print(f"Error: Unable to read frame {idx}.")

    # Release the video capture object
    cap.release()
    return frame

# Example usage
video_path = "test.avi"
output_folder = "output_frames"
frame_indices = [0, 1, 2]  # Specify the indices of frames you want to extract

# Call the function to extract frames
frames = extract_frames(video_path, frame_indices)
print(frames[0].shape)
# print(frames[0])


(1280, 3)


In [11]:
def train(x_train, x_test, y_train, y_test):

    x_test = x_test.view(-1, 3, 720, 1280)
    x_train = x_train.view(-1, 3, 720, 1280)

    y_train = y_train.long()
    y_test = y_test.long()

    model = Network()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=.2)

    # create the train dataset and loader (following pytorch documentation)
    train_dataset = TensorDataset(x_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

    epochs = 5
    for epoch in range(epochs):
        n = 0
        for inputs, labels in train_loader:
            inputs = inputs.view(-1, 3, 720, 1280)
            labels = labels.view(-1)
            # print("labels: ", labels)
            optimizer.zero_grad()

            outputs = model(inputs.float())
            #print(outputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            print("Step: "+str(n))
            n += 1
        print("epoch: ",str(epoch))
    

    num_test_samples = 32
    train_indices = random.sample(range(len(x_train)), num_test_samples)
    train_inputs = x_train[train_indices]
    train_labels = y_train[train_indices]

    test_outputs = model(x_test.float())
    _, test_predicted = torch.max(test_outputs, 1)
    test_accuracy = (test_predicted == y_test).sum().item() / len(y_test)

    train_outputs = model(train_inputs.float())
    _, train_predicted = torch.max(train_outputs, 1)
    train_accuracy = (train_predicted ==
                        train_labels).sum().item() / len(train_labels)
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}, Test Accuracy: {test_accuracy * 100:.2f}%, Train Accuracy: {train_accuracy * 100:.2f}%')
    torch.save(model.state_dict(), "model/model2.pth")
    return model


In [12]:
def test():
    print()
    

In [13]:
import os
### The [0] gets the last frame, [5] would get 5th to last, etc.
path = "./0strike_avis"
files = os.listdir(path)

strikes = []
for file in files:
    strikes.append(extract_frames(path+"/"+file, [0]))

path = "./0ball_avis"
files = os.listdir(path)

balls = []
for file in files:
    balls.append(extract_frames(path+"/"+file, [0]))


In [14]:
b_idx = int(len(balls)*.9)
s_idx = int(len(strikes)*.9)

print(b_idx)
print(s_idx)

x_train = balls[:b_idx] + strikes[:s_idx]
x_train = torch.tensor(np.array(x_train))
x_train = torch.squeeze(x_train)

x_test = balls[b_idx:] + strikes[s_idx:]
x_test = torch.tensor(np.array(x_test))
x_test = torch.squeeze(x_test)

y_train = torch.tensor(np.concatenate((np.zeros( b_idx), np.ones(s_idx))))
y_test = torch.tensor(np.concatenate((np.zeros(len(balls)-b_idx), np.ones(len(strikes)-s_idx))))

print(x_test.shape)
print(x_train.shape)

print(y_test.shape)
print(y_train.shape)

348
215
torch.Size([63, 720, 1280, 3])
torch.Size([563, 720, 1280, 3])
torch.Size([63])
torch.Size([563])


In [15]:
print(y_train.shape)

torch.Size([563])


In [16]:
train(x_train, x_test, y_train, y_test)

Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
epoch:  0
Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
epoch:  1
Step: 0
Step: 1
Step: 2
Step: 3
Step: 4
Step: 5
Step: 6
Step: 7
Step: 8
Step: 9
Step: 10
Step: 11
Step: 12
Step: 13
Step: 14
Step: 15
Step: 16
Step: 17
Step: 18
Step: 19
Step: 20
Step: 21
Step: 22
Step: 23
Step: 24
Step: 25
Step: 26
Step: 27
Step: 28
Step: 29
Step: 30
Step: 31
Step: 32
Step: 33
Step: 34
Step: 35
epoch:  2
Step: 0
Step: 1
Step: 2
Step

Network(
  (conv1): Conv2d(3, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(32, 16, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (bn5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv6): Conv2d(16, 4, kernel_size=(3, 3), stride=(2, 2), padding=(2, 2))
  (bn6): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=3772, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=2, bias=True)
)

**Start of Pitch-Type Classification**

In [24]:
import csv

def get_files(pitchtype):
    pitchnames = []
    with open('./pitch_types/'+pitchtype+'.csv', 'r') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            pitchnames.append(row['Title'])

    return(pitchnames)



def get_pitches(filenames):
    path = "./0strike_avis"
    files = os.listdir(path)

    pitches = []
    for file in files:
        if file in filenames:
            pitches.append(extract_frames(path+"/"+file, [0]))

    path = "./0ball_avis"
    files = os.listdir(path)

    for file in files:
        if file in filenames:
            pitches.append(extract_frames(path+"/"+file, [0]))

    return pitches

# All pitch types can be implemented the same as above, 
# path should include both 0strike_avis and 0ball_avis
curvenames = get_files('curves')
curves = get_pitches(curvenames)

changenames = get_files("changes")
changes = get_pitches(changenames)

fastnames = get_files("fastballs")
fastballs = get_pitches(fastnames)

slidernames = get_files("sliders")
sliders = get_pitches(slidernames)


curve_idx = int(len(curves)*.8)
slider_idx = int(len(sliders)*.8)
fast_idx = int(len(fastballs)*.8)
change_idx = int(len(changes)*.8)

x_train = curves[:curve_idx] + sliders[:slider_idx]+ fastballs[:fast_idx] + changes[:change_idx]
x_train = torch.tensor(np.array(x_train))
x_train = torch.squeeze(x_train)

x_test = curves[curve_idx:] + sliders[slider_idx:]+fastballs[fast_idx:] + changes[change_idx:]
x_test = torch.tensor(np.array(x_test))
x_test = torch.squeeze(x_test)

y_train = torch.tensor(np.concatenate((np.zeros( curve_idx), np.ones(slider_idx), 2*np.ones(fast_idx), 3*np.ones(change_idx))))
y_test = torch.tensor(np.concatenate((np.zeros(len(curves)-curve_idx), np.ones(len(sliders)-slider_idx), 2*np.ones(len(fastballs)-fast_idx), 3*np.ones(len(changes)-change_idx))))

